In [21]:
import numpy as np 
import pandas as pd
import pandas.util.testing as tm
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [22]:
flights_df = pd.read_csv('./data/flights.csv')

C:\Users\Jesse\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (25) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


 CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY

In [8]:
flights_df['cancellation_code'].notna().sum()

269208

In [9]:
flights_df['cancellation_code'].unique()

array([nan, 'B', 'C', 'A', 'D', 'X'], dtype=object)

In [17]:
flights_df.loc[(flights_df['carrier_delay'].notna()), ['carrier_delay', 'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay', 'dep_delay', 'arr_delay', ]]

,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,dep_delay,arr_delay
1,39.0,0.0,0.0,0.0,0.0,53.0,39.0
3,18.0,0.0,0.0,0.0,0.0,19.0,18.0
6,3.0,0.0,28.0,0.0,0.0,3.0,31.0
7,0.0,0.0,0.0,0.0,15.0,15.0,15.0
9,11.0,0.0,4.0,0.0,4.0,15.0,19.0
...,...,...,...,...,...,...,...
15927470,0.0,0.0,15.0,0.0,0.0,-1.0,15.0
15927472,298.0,0.0,0.0,0.0,0.0,330.0,298.0
15927474,0.0,0.0,70.0,0.0,0.0,81.0,70.0
15927481,9.0,0.0,15.0,0.0,0.0,9.0,24.0


In [18]:
flights_df.loc[:,['arr_delay', 'dep_delay']]

,arr_delay,dep_delay
0,6.0,0.0
1,39.0,53.0
2,-17.0,-9.0
3,18.0,19.0
4,-10.0,-3.0
...,...,...
15927480,-16.0,-1.0
15927481,24.0,9.0
15927482,-26.0,-11.0
15927483,-13.0,11.0


### Things to note:

Assignment

=================================================================================


"We are going to work on three different types of Machine Learning problem:

1. Regression Problem: The goal is to predict delay of flights.
2. Multiclass Classification: If the plane was delayed, we will predict what type of delay it is (will be).
3. Binary Classification: The goal is to predict if the flight will be cancelled."



==================================================================================


- flights_test CSV contain all the columns that we can use for our machine learning model
- Dont drop Cancelled column as that will be used in binary classification. Update: Has no missing values
- tail_num column is used in our test set but has missing values, but not >40%, fill with -1
- crs_elapsed_time column has 20 missing values but is used in ML dataset, drop all rows with missing values.
- Feature engineer distances between cities. Use Geomap API?
- Using flight_date we can create new column for season?
- Create a new column with the weather for that departing city
- Create column with weather for cities for sample data
- dept_delay (minutes): Difference in minutes between scheduled and actual departure time. Early departures show negative numbers.This is our assignment prediction and regression label, drop all rows with NaN values. There are 263754 missing values, therefore those rows to be dropped
- arr_delay: Difference in minutes between scheduled and actual arrival time. Early arrivals show negative numbers. Predicting arr_delay is not part of the regression task (assignment), but we can make a model for that too. There are 311744 missig values therefore those rows to be dropped.



In [ ]:
cols_ML = ['fl_date', 
           'mkt_unique_carrier', 
           'branded_code_share', 
           'mkt_carrier', 
           'mkt_carrier_fl_num', 
           'op_unique_carrier', 
           'tail_num', 
           'op_carrier_fl_num', 
           'origin_airport_id', 
           'origin', 
           'origin_city_name', 
           'dest_airport_id', 
           'dest', 
           'dest_city_name', 
           'crs_dep_time', 
           'crs_arr_time', 
           'dup', 
           'crs_elapsed_time', 
           'flights', 
           'distance']

### Things do to:

- <s>check for missing values drop columns with greater then 40% missing values </s>
- <s>drop mkt_unique_carrier because mkt_carrier contains the exact same value</s>
- <s>drop cancellation_code (>40% missing) and dup (same value) </s> Update: No missing values for 'Cancelled', but dropped dup because only 1 unique value
- <s>crs_dep_time, dep_time, crs_arr_time, arr_time, wheels_off, wheels_on, crs_arr_time, arr_time convert to datetime (hhmm)</s> Update: Missing values were filled with '00:00' 
- <s>fl_date parse to datetime (yyyy-mm-dd)</s>
- <s>Fill tail_num column with 0</s>
- arr_delay is the regression label, drop all rows with empty arr_delay
- Create new column for Multiclassification Whether it is a departal delay or a arrival delay
- Create month and year column using fl_date

In [15]:
flights_df.shape

(15927485, 42)

In [16]:
flights_df.head()

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name
0,2019-08-22,B6,B6,B6,984,B6,N281JB,984,14492,RDU,...,612,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-08-22,B6,B6,B6,987,B6,N978JB,987,10721,BOS,...,2611,39.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
2,2019-08-22,B6,B6,B6,988,B6,N983JT,988,12892,LAX,...,2611,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-08-22,B6,B6,B6,989,B6,N339JB,989,10721,BOS,...,399,18.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
4,2019-08-22,B6,B6,B6,990,B6,N583JB,990,13204,MCO,...,944,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
flights_df.isna().sum()

fl_date                       0
mkt_unique_carrier            0
branded_code_share            0
mkt_carrier                   0
mkt_carrier_fl_num            0
op_unique_carrier             0
tail_num                  49269
op_carrier_fl_num             0
origin_airport_id             0
origin                        0
origin_city_name              0
dest_airport_id               0
dest                          0
dest_city_name                0
crs_dep_time                  0
dep_time                 258814
dep_delay                263754
taxi_out                 273274
wheels_off               273264
wheels_on                281162
taxi_in                  281172
crs_arr_time                  0
arr_time                 275079
arr_delay                311744
cancelled                     0
cancellation_code      15658277
diverted                      0
dup                           0
crs_elapsed_time             20
actual_elapsed_time      309157
air_time                 315221
flights 

In [ ]:
cols_ML = ['fl_date', 
           'mkt_unique_carrier', 
           'branded_code_share', 
           'mkt_carrier', 
           'mkt_carrier_fl_num', 
           'op_unique_carrier', 
           'tail_num', 
           'op_carrier_fl_num', 
           'origin_airport_id', 
           'origin', 
           'origin_city_name', 
           'dest_airport_id', 
           'dest', 
           'dest_city_name', 
           'crs_dep_time', 
           'crs_arr_time', 
           'dup', 
           'crs_elapsed_time', 
           'flights', 
           'distance']

In [22]:
0.4*flights_df.shape[0]

6370994.0

In [24]:
object_columns = flights_df.select_dtypes('object')

In [26]:
object_columns.head(3)

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,op_unique_carrier,tail_num,origin,origin_city_name,dest,dest_city_name,cancellation_code,dup
0,2019-08-22,B6,B6,B6,B6,N281JB,RDU,"Raleigh/Durham, NC",BOS,"Boston, MA",NaN,N
1,2019-08-22,B6,B6,B6,B6,N978JB,BOS,"Boston, MA",LAX,"Los Angeles, CA",NaN,N
2,2019-08-22,B6,B6,B6,B6,N983JT,LAX,"Los Angeles, CA",BOS,"Boston, MA",NaN,N


In [27]:
# mkt_unique_carrier unique values
# 11 items

flights_df['mkt_unique_carrier'].unique()

array(['B6', 'DL', 'AA', 'UA', 'NK', 'WN', 'G4', 'HA', 'F9', 'AS', 'VX'],
      dtype=object)

In [28]:
# branded_code_share unique values

flights_df['branded_code_share'].unique()

array(['B6', 'DL_CODESHARE', 'AA', 'DL', 'UA_CODESHARE', 'NK',
       'AA_CODESHARE', 'WN', 'G4', 'HA', 'HA_CODESHARE', 'F9', 'AS',
       'AS_CODESHARE', 'UA', 'VX'], dtype=object)

In [38]:
# mkt_carrier unique values
# 11 items


flights_df['mkt_carrier'].unique()

# We see that its same values as mkt_unique_carrier unique values

array(['B6', 'DL', 'AA', 'UA', 'NK', 'WN', 'G4', 'HA', 'F9', 'AS', 'VX'],
      dtype=object)

In [60]:
# mkt_carrier and mkt_unique_carrier match for every row therefore we can drop one or the other

flights_df.loc[flights_df['mkt_carrier'] != flights_df['mkt_unique_carrier']]

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name


In [39]:
# 28 items
flights_df['op_unique_carrier'].unique()

array(['B6', '9E', 'AA', 'DL', 'YV', 'NK', 'MQ', 'OH', 'WN', 'G4', 'YX',
       'C5', 'ZW', 'AX', 'EV', 'PT', 'HA', 'CP', 'EM', 'F9', 'AS', 'QX',
       'OO', 'UA', 'G7', 'KS', '9K', 'VX'], dtype=object)

In [40]:
# 'op_unique_carrier' contains everything in 'mkt_carrier'/'mkt_unique_carrier' plus 17 extra items

# The extra items in 'op_unique_carrier'
set(flights_df['op_unique_carrier'].unique()).difference(flights_df['mkt_carrier'].unique())

{'9E',
 '9K',
 'AX',
 'C5',
 'CP',
 'EM',
 'EV',
 'G7',
 'KS',
 'MQ',
 'OH',
 'OO',
 'PT',
 'QX',
 'YV',
 'YX',
 'ZW'}

In [50]:
# 6488 unique items (Represents unique plane?)
# Has alot of missing values?

flights_df['tail_num'].unique().shape

(6488,)

In [62]:
# Origin and destination codes equal

print('Equal length of unique values for "origin" and "dest": ',flights_df['origin'].unique().shape[0] == flights_df['dest'].unique().shape[0])


# Both origin and desitnation contain the same codes

print('Checking for different codes between "origin" and "dest" columns: ', len(set(flights_df['origin'].unique()) - set(flights_df['dest'].unique())))

Equal length of unique values for "origin" and "dest":  True
Checking for different codes between "origin" and "dest" columns:  0


In [63]:
# Keep 'origin_city_name' and 'dest_city_name' for later use, may drop them for machine learning model
flights_df['origin_city_name'].unique()

array(['Raleigh/Durham, NC', 'Boston, MA', 'Los Angeles, CA',
       'Orlando, FL', 'Tampa, FL', 'Denver, CO', 'Atlanta, GA',
       'Anchorage, AK', 'Buffalo, NY', 'Long Beach, CA', 'Seattle, WA',
       'Chicago, IL', 'New York, NY', 'Jacksonville, FL',
       'Fort Lauderdale, FL', 'Washington, DC', 'San Juan, PR',
       'Charlotte Amalie, VI', 'Austin, TX', 'Savannah, GA',
       'West Palm Beach/Palm Beach, FL', 'Fort Myers, FL',
       'Providence, RI', 'Las Vegas, NV', 'Portland, OR',
       'Charleston, SC', 'Hartford, CT', 'Richmond, VA', 'Phoenix, AZ',
       'Nantucket, MA', 'Dallas/Fort Worth, TX', "Martha's Vineyard, MA",
       'Baltimore, MD', 'San Francisco, CA', 'Charlotte, NC',
       'White Plains, NY', 'Newark, NJ', 'Pittsburgh, PA', 'Detroit, MI',
       'New Orleans, LA', 'Burlington, VT', 'Minneapolis, MN',
       'Philadelphia, PA', 'Nashville, TN', 'Syracuse, NY',
       'Cleveland, OH', 'Worcester, MA', 'San Jose, CA', 'Burbank, CA',
       'Portland, ME', 'M

In [66]:
flights_df['cancellation_code'].value_counts()

B    142089
A     70569
C     56484
D        64
X         2
Name: cancellation_code, dtype: int64

In [70]:
# Will drop, this column contains more then 40% missing values

flights_df['cancellation_code'].isna().sum()

15658277

In [67]:
object_columns.head()

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,op_unique_carrier,tail_num,origin,origin_city_name,dest,dest_city_name,cancellation_code,dup
0,2019-08-22,B6,B6,B6,B6,N281JB,RDU,"Raleigh/Durham, NC",BOS,"Boston, MA",NaN,N
1,2019-08-22,B6,B6,B6,B6,N978JB,BOS,"Boston, MA",LAX,"Los Angeles, CA",NaN,N
2,2019-08-22,B6,B6,B6,B6,N983JT,LAX,"Los Angeles, CA",BOS,"Boston, MA",NaN,N
3,2019-08-22,B6,B6,B6,B6,N339JB,BOS,"Boston, MA",DCA,"Washington, DC",NaN,N
4,2019-08-22,B6,B6,B6,B6,N583JB,MCO,"Orlando, FL",JFK,"New York, NY",NaN,N


In [71]:
# contains 1 value, will drop 
flights_df['dup'].unique()

array(['N'], dtype=object)

#### Numeric Columns

In [80]:
flights_df.dtypes.unique()

array([dtype('O'), dtype('int64'), dtype('float64')], dtype=object)

In [83]:
numerical_columns = flights_df.select_dtypes(['int64', 'float64'])
numerical_columns.head()

,mkt_carrier_fl_num,op_carrier_fl_num,origin_airport_id,dest_airport_id,crs_dep_time,dep_time,dep_delay,taxi_out,wheels_off,wheels_on,...,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name
0,984,984,14492,10721,1501,1501.0,0.0,25.0,1526.0,1656.0,...,612,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,987,987,10721,12892,2150,2243.0,53.0,14.0,2257.0,140.0,...,2611,39.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
2,988,988,12892,10721,2350,2341.0,-9.0,24.0,5.0,802.0,...,2611,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,989,989,10721,11278,1814,1833.0,19.0,25.0,1858.0,2011.0,...,399,18.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
4,990,990,13204,12478,618,615.0,-3.0,15.0,630.0,841.0,...,944,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [84]:
flights_df['mkt_carrier_fl_num'].unique().shape

(7226,)

In [85]:
flights_df['op_carrier_fl_num'].unique().shape

(7252,)

In [88]:
print('Equal length of unique values for "origin_airport_id" and "dest_airport_id": ',flights_df['origin_airport_id'].unique().shape[0] == flights_df['dest_airport_id'].unique().shape[0])
print('Checking for different codes between "origin_airport_id" and "dest_airport_id" columns: ', len(set(flights_df['origin_airport_id'].unique()) - set(flights_df['dest_airport_id'].unique())))

Equal length of unique values for "origin_airport_id" and "dest_airport_id":  True
Checking for different codes between "origin_airport_id" and "dest_airport_id" columns:  0


In [91]:
# CRS Departure Time (local time: hhmm) 
# Convert to datetime format

flights_df['crs_dep_time'].isna().sum()

0

In [94]:
#  Actual Departure Time (local time: hhmm)
# Convert to datetime format

flights_df['dep_time'].isna().sum()

258814

In [98]:
# Difference in MINUTES between scheduled and actual departure time. Early departures show negative numbers.

flights_df['dep_delay']

0             0.0
1            53.0
2            -9.0
3            19.0
4            -3.0
            ...  
15927480     -1.0
15927481      9.0
15927482    -11.0
15927483     11.0
15927484    217.0
Name: dep_delay, Length: 15927485, dtype: float64

In [95]:
# Taxi Out Time, in Minutes

flights_df['taxi_out']

0           25.0
1           14.0
2           24.0
3           25.0
4           15.0
            ... 
15927480    12.0
15927481    36.0
15927482    29.0
15927483    17.0
15927484    10.0
Name: taxi_out, Length: 15927485, dtype: float64

In [100]:
# Difference in MINUTES between scheduled and actual arrival time. Early arrivals show negative numbers.

flights_df['arr_delay']

0             6.0
1            39.0
2           -17.0
3            18.0
4           -10.0
            ...  
15927480    -16.0
15927481     24.0
15927482    -26.0
15927483    -13.0
15927484    185.0
Name: arr_delay, Length: 15927485, dtype: float64

In [103]:
# Imbalence in the data, Can use SMOTE to rebalance and test

flights_df['cancelled'].value_counts()

0    15658277
1      269208
Name: cancelled, dtype: int64

In [106]:
# Diverted Flight Indicator (1=Yes)

flights_df['diverted'].value_counts()

0    15887546
1       39939
Name: diverted, dtype: int64

In [108]:
#  Flight Time, in Minutes

flights_df['air_time']

0            90.0
1           343.0
2           297.0
3            73.0
4           131.0
            ...  
15927480    143.0
15927481    162.0
15927482    149.0
15927483     91.0
15927484    122.0
Name: air_time, Length: 15927485, dtype: float64

In [109]:
flights_df['actual_elapsed_time']

0           121.0
1           365.0
2           335.0
3           105.0
4           155.0
            ...  
15927480    166.0
15927481    205.0
15927482    182.0
15927483    112.0
15927484    137.0
Name: actual_elapsed_time, Length: 15927485, dtype: float64

In [110]:
flights_df['crs_elapsed_time']

0           115.0
1           379.0
2           343.0
3           106.0
4           162.0
            ...  
15927480    181.0
15927481    190.0
15927482    197.0
15927483    136.0
15927484    169.0
Name: crs_elapsed_time, Length: 15927485, dtype: float64

# Actual Data Cleaning

#### Check if any rows contains ALL NaN values

In [23]:
flights_df.loc[flights_df.isna().all(1)]

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name


#### Check if column contains >40% missing values, drop that column 

Also checking to ensure the column to be dropped is NOT within our flights_test evaluation dataframe

In [25]:
flights_test = pd.read_csv('./data/flights_test.csv')

flights_test_cols = flights_test.columns

In [26]:
flights_df.shape

(15927485, 42)

In [27]:
# Dropping all columns if they have >40% missing values and those columns are in flights_test_cols (we need them for ML)

total_rows = flights_df.shape[0] # 15927485
threshold_col_num = total_rows*0.4 # 6370994

# Therefore if a columns NaN values are >6370994 then DROP that column

# From counting should drop 10 columns, therefore end with 32 columns at the end
count = 0
for col in flights_df.columns:
    if col not in flights_test_cols:
        total_na = flights_df[col].isna().sum()
        if total_na > threshold_col_num:
            flights_df.drop(col, axis=1, inplace=True)
            print('Column dropped: ', col)
            count += 1
            
            
print('Number of columns dropped: ', count)

Column dropped:  cancellation_code
Column dropped:  carrier_delay
Column dropped:  weather_delay
Column dropped:  nas_delay
Column dropped:  security_delay
Column dropped:  late_aircraft_delay
Column dropped:  first_dep_time
Column dropped:  total_add_gtime
Column dropped:  longest_add_gtime
Column dropped:  no_name
Number of columns dropped:  10


In [28]:
# Should have 32 columns
flights_df.shape

(15927485, 32)

#### Dropping mkt_unique_code_carrier columns because mkt_carrier contains exact same value within same row

In [29]:
# Should have 31 columns afterwards 
flights_df.drop('mkt_unique_carrier', axis=1, inplace=True)

flights_df.shape

(15927485, 31)

#### Dropping 'dup' column, only contains 1 value across entire column

In [30]:
# Should only have 30 columnes afterwards

flights_df.drop('dup', axis=1, inplace=True)

flights_df.shape

(15927485, 30)

In [4]:
# Save csv file 

# flights_df.to_csv('./data/flights_df_cleaned.csv', index=False)

NameError: name 'flights_df' is not defined

In [63]:
flights_df = pd.read_csv('./data/flights_df_cleaned.csv', index_col=0)

flights_df.head()

C:\Users\Jesse\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,fl_date,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,...,crs_arr_time,arr_time,arr_delay,cancelled,diverted,crs_elapsed_time,actual_elapsed_time,air_time,flights,distance
0,2019-08-22,B6,B6,984,B6,N281JB,984,14492,RDU,"Raleigh/Durham, NC",...,1656,1702.0,6.0,0,0,115.0,121.0,90.0,1,612
1,2019-08-22,B6,B6,987,B6,N978JB,987,10721,BOS,"Boston, MA",...,109,148.0,39.0,0,0,379.0,365.0,343.0,1,2611
2,2019-08-22,B6,B6,988,B6,N983JT,988,12892,LAX,"Los Angeles, CA",...,833,816.0,-17.0,0,0,343.0,335.0,297.0,1,2611
3,2019-08-22,B6,B6,989,B6,N339JB,989,10721,BOS,"Boston, MA",...,2000,2018.0,18.0,0,0,106.0,105.0,73.0,1,399
4,2019-08-22,B6,B6,990,B6,N583JB,990,13204,MCO,"Orlando, FL",...,900,850.0,-10.0,0,0,162.0,155.0,131.0,1,944


#### Converting certain columns to datetime

convert to datetime (hhmm)

- crs_dep_time 
- dep_time
- crs_arr_time 
- arr_time 
- wheels_off 
- wheels_on

In [31]:
cols = ['crs_dep_time', 'dep_time', 'crs_arr_time', 'arr_time', 'wheels_off', 'wheels_on']

flights_df.loc[:, cols].dtypes

crs_dep_time      int64
dep_time        float64
crs_arr_time      int64
arr_time        float64
wheels_off      float64
wheels_on       float64
dtype: object

In [33]:
def convert_time_to_string(x):
    if np.isnan(x):
        return '00:00'
    
    if x == 2400:
        return '23:59'
    
    string = str(int(x))
    
    if len(string) < 4:
        for _ in range(4-len(string)):
            string = '0' + string
    
    return string[:2] + ':' + string[2:]

In [34]:
# example
convert_time_to_string(1)

'00:01'

In [35]:
# Filling NaN values with '00:00' and converting ints/floats (hhmm) with datetime objects

flights_df['crs_dep_time'] = flights_df['crs_dep_time'].apply(lambda x: convert_time_to_string(x))
flights_df['crs_dep_time'] = pd.to_datetime(flights_df['crs_dep_time'], format = '%H:%M')

flights_df['dep_time'] = flights_df['dep_time'].apply(lambda x: convert_time_to_string(x))
flights_df['dep_time'] = pd.to_datetime(flights_df['dep_time'], format = '%H:%M')

flights_df['crs_arr_time'] = flights_df['crs_arr_time'].apply(lambda x: convert_time_to_string(x))
flights_df['crs_arr_time'] = pd.to_datetime(flights_df['crs_arr_time'], format = '%H:%M')

flights_df['arr_time'] = flights_df['arr_time'].apply(lambda x: convert_time_to_string(x))
flights_df['arr_time'] = pd.to_datetime(flights_df['arr_time'], format = '%H:%M')

flights_df['wheels_off'] = flights_df['wheels_off'].apply(lambda x: convert_time_to_string(x))
flights_df['wheels_off'] = pd.to_datetime(flights_df['wheels_off'], format = '%H:%M')

flights_df['wheels_on'] = flights_df['wheels_on'].apply(lambda x: convert_time_to_string(x))
flights_df['wheels_on'] = pd.to_datetime(flights_df['wheels_on'], format = '%H:%M')

In [36]:
flights_df.loc[:, cols].isna().sum()

crs_dep_time    0
dep_time        0
crs_arr_time    0
arr_time        0
wheels_off      0
wheels_on       0
dtype: int64

In [37]:
flights_df[cols].dtypes

crs_dep_time    datetime64[ns]
dep_time        datetime64[ns]
crs_arr_time    datetime64[ns]
arr_time        datetime64[ns]
wheels_off      datetime64[ns]
wheels_on       datetime64[ns]
dtype: object

In [38]:
# Save csv file 

# flights_df.to_csv('./data/flights_df_cleaned.csv', index=False)

In [ ]:
# flights_df = pd.read_csv('./data/flights_df_cleaned.csv')

### Things do to:

- <s>check for missing values drop columns with greater then 40% missing values </s>
- <s>drop mkt_unique_carrier because mkt_carrier contains the exact same value</s>
- <s>drop cancellation_code (>40% missing) and dup (same value) </s> Update: No missing values for 'Cancelled', but dropped dup because only 1 unique value
- <s>crs_dep_time, dep_time, crs_arr_time, arr_time, wheels_off, wheels_on, crs_arr_time, arr_time convert to datetime (hhmm)</s> Update: Missing values were filled with '00:00' 
- fl_date parse to datetime (yyyy-mm-dd)
- Fill tail_num column with -1
- Drop all rows with missing values in crs_elapsed_time (Regression label)
- Create new column for Multiclassification Whether it is a departal delay or a arrival delay

In [46]:
flights_df = pd.read_csv('./data/flights_df_cleaned.csv')

#### Convert fl_date column to datetime

In [47]:
flights_df['fl_date'][0]

'2019-08-22'

In [48]:
flights_df['fl_date'] = pd.to_datetime(flights_df['fl_date'], format='%Y-%m-%d')

In [49]:
flights_df['fl_date'].dtype

dtype('<M8[ns]')

#### Fill missing tail_num with 0

In [53]:
flights_df['tail_num'].isna().sum()

49269

In [57]:
flights_df['tail_num'].fillna(0, inplace=True)

In [58]:
flights_df['tail_num'].isna().sum()

0